In [1]:
import numpy as np
import tensorflow as tf 
from collections import deque
import random
import pdb
from replay_memory import ReplayMemory # AC replay memory
from AC_networks_tf import ActorNet, CriticNet
from test_game import test_env
import os
from __future__ import print_function
from info_obj import info_holder

In [2]:
import gym
#game_name = 'LunarLander-v2'
game_name = 'CartPole-v0'
#game_name = 'MountainCar-v0'
game = gym.make(game_name)
state_dim = game.observation_space.shape[0]
action_dim = game.action_space.n
'''
game = test_env()
state_dim = 1
action_dim = 2
'''

[2016-12-31 01:36:47,408] Making new env: CartPole-v0


'\ngame = test_env()\nstate_dim = 1\naction_dim = 2\n'

In [9]:
# info = graph, dim_s, dim_a, hid_layers, learning_rate, tau
save_dir = './Results/'+ game_name[:-3] + '/net/'
if 'LunarLander' in game_name:
    # BUFFER_SIZE = int(1e5)
    # MINIBATCH_SIZE = 256
    GAMMA = 0.99
    LEARNING_RATE = 1. #5e-5 lr/2
    lr = LEARNING_RATE # / MINIBATCH_SIZE
    lr_decay_ep = 100
    lr_decay_rate = 0.955
    eps_decay_rate = 0.955
    TAU = 0
    ENTROPY_BETA = 0
    EPSILON = 1
    C_update = 100 #episode
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(1e3)
    print_ep = int(1e2)
    save_ep = int(1e3)
    
elif 'CartPole' in game_name:
    # BUFFER_SIZE = int(1e5)
    # MINIBATCH_SIZE = 128
    GAMMA = 0.99
    LEARNING_RATE = 1e-4 #5e-5 lr/2
    lr = LEARNING_RATE # / MINIBATCH_SIZE
    lr_decay_ep = 100
    lr_decay_rate = 1
    eps_decay_rate = 0.9
    TAU = 0
    ENTROPY_BETA = 0
    EPSILON = 1
    C_update = 16 #episode
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(2e2)
    print_ep = int(1e2)
    save_ep = int(1e3)
    
elif 'MountainCar' in game_name:
    # BUFFER_SIZE = int(1e5)
    # MINIBATCH_SIZE = 256
    GAMMA = 0.99
    LEARNING_RATE = 1e-2 #5e-5 lr/2
    lr = LEARNING_RATE # / MINIBATCH_SIZE
    lr_decay_ep = 100
    lr_decay_rate = 0.955
    eps_decay_rate = 0.955
    TAU = 0
    ENTROPY_BETA = 0
    EPSILON = 1
    C_update = 100 #episode
    MAX_EPISODE = int(1e4)
    MAX_STEP = int(2e2)
    print_ep = int(1e2)
    save_ep = int(1e3)

buff = ReplayMemory()
info = info_holder()
try:
    actor
except:
    pass
else:
    actor.reset_net()
try:
    critic
except:
    pass
else:
    critic.reset_net()  
    
graph = tf.Graph()
with graph.as_default():
    info.graph = graph
    info.dim_s = state_dim
    info.dim_a = action_dim
    info.act_hid_layers = np.array([16])
    info.crit_hid_layers = np.array([16])
    info.learning_rate = LEARNING_RATE
    info.tau = TAU
    info.entropy_beta = ENTROPY_BETA
    actor = ActorNet(info)
    critic = CriticNet(info)

In [13]:
# lr = LEARNING_RATE
buff.clear()
last_ep = 0
start_update = 0
with tf.Session(graph=graph) as sess:
    actor.set_up(sess)
    critic.set_up(sess)
    sess.run(tf.global_variables_initializer())
    
    ep_reward = np.zeros(MAX_EPISODE)
    ep_success = np.zeros(MAX_EPISODE)
    ep_crash = np.zeros(MAX_EPISODE)
    #batch_loss = np.zeros(MAX_EPISODE)
    avg_var = 0
    
    for ep in range(MAX_EPISODE):
        s = game.reset()
        var = 0
        for st in range(MAX_STEP):
            a = actor.take_action(s.reshape(-1,info.dim_s))
            s2, r, terminal, report = game.step(a)

            buff.add(np.reshape(s, (info.dim_s,)).astype(np.float32), np.array(a).astype(np.float32),\
                     np.array(r).astype(np.float32), np.reshape(s2, (info.dim_s,)).astype(np.float32),\
                     np.array(terminal).astype(np.float32))

            ep_reward[ep] += (1.*r)
            var += np.sum(actor.give_policy(s.reshape(-1,info.dim_s))**2)
            s = s2

            if terminal:
                var = var/st
                avg_var += (var/print_ep)
                break
                
        if ((ep % C_update) == (C_update -1)):
            (s_batch, a_batch, r_batch, s2_batch, t_batch) = \
                buff.sample_batch()
            s_batch = np.array(s_batch).reshape(-1, info.dim_s)
            s2_batch = np.array(s2_batch).reshape(-1, info.dim_s)
            a_batch = np.array(a_batch)#.reshape(-1, 1)
            r_batch = np.array(r_batch)#.reshape(-1, 1)
            t_batch = np.array(t_batch)#.reshape(-1, 1)

            Q_2 = critic.predict(s2_batch)
            Q_2 = Q_2[:,0]
            
            R_batch = r_batch + GAMMA * Q_2 * (1-t_batch)
            R_batch = R_batch.reshape(-1, 1)
            #pdb.set_trace()
            predicted_Q, _ = critic.train(s_batch, R_batch, lr)
            td_batch = R_batch - predicted_Q
            actor.train(s_batch, a_batch, td_batch, lr)
                
        if terminal == False:
            var = var/MAX_STEP
            avg_var += (var/print_ep)
        if (r >= 100):
            ep_success[ep] = 1
        elif (r <= -100):
            ep_crash[ep] = 1
        
        if ((ep % print_ep) == (print_ep-1)) or (ep == 0):
            print('Ep: %d --- Ave R: %.1f  |  Suc: %.2f  |  crash: %.2f  |  Max R: %.1f  |  var: %.2f  |  lr:%.2fe-4'\
                  %(ep, np.mean(ep_reward[last_ep:ep+1]), np.mean(ep_success[last_ep:ep+1]), \
                    np.mean(ep_crash[last_ep:ep+1]), np.amax(ep_reward[last_ep:ep+1]), var, 1e4*lr))
            last_ep = ep
            avg_var = 0
        
        if (ep == 0):
            if (os.path.isdir(save_dir)):
                raise NameError('Directory net exists')
                
        if (ep == (save_ep-1)):
            os.mkdir(save_dir)
            with open(save_dir + 'params.txt', 'w') as f:
                f.write('Learning rate = %f\n' %LEARNING_RATE)
                f.write('lr decay = %f\n' %lr_decay_rate)
                f.write('hidden layers: %d\n' %info.hid_layers)
                f.write('update cycle: %d\n' %C_update)
                
        if (ep % save_ep) == (save_ep-1):
            print(' ***** Saving...  at episode %d ...' %ep, end="")
            saver = tf.train.Saver()
            filename = save_dir + 'Episod_' + str(ep) + '.chk'
            saver.save(sess, filename)
            with open(save_dir + 'rewards.txt', 'w') as f:
                f.write('%d, , , , ,Learning rate = %f - lr decay = %f - C_update = %d - hidden layers: %d\n' \
                        %(ep, LEARNING_RATE, lr_decay_rate, C_update, info.hid_layers) )
                for i in range(ep):
                    f.write('%f\n' %(ep_reward[i]))
            print(' Done ******')
            
        if (ep % lr_decay_ep) == (lr_decay_ep-1):
            lr = lr_decay_rate * lr
    
print('DONE') 
    

Ep: 0 --- Ave R: 34.0  |  Suc: 0.00  |  crash: 0.00  |  Max R: 34.0  |  var: 0.62  |  lr:1.00e-4
Ep: 99 --- Ave R: 25.4  |  Suc: 0.00  |  crash: 0.00  |  Max R: 65.0  |  var: 0.65  |  lr:1.00e-4
Ep: 199 --- Ave R: 26.2  |  Suc: 0.00  |  crash: 0.00  |  Max R: 57.0  |  var: 0.65  |  lr:1.00e-4
Ep: 299 --- Ave R: 26.2  |  Suc: 0.00  |  crash: 0.00  |  Max R: 59.0  |  var: 0.60  |  lr:1.00e-4
Ep: 399 --- Ave R: 26.0  |  Suc: 0.00  |  crash: 0.00  |  Max R: 76.0  |  var: 0.72  |  lr:1.00e-4
Ep: 499 --- Ave R: 24.7  |  Suc: 0.00  |  crash: 0.00  |  Max R: 54.0  |  var: 0.68  |  lr:1.00e-4
Ep: 599 --- Ave R: 25.2  |  Suc: 0.00  |  crash: 0.00  |  Max R: 60.0  |  var: 0.67  |  lr:1.00e-4
Ep: 699 --- Ave R: 24.9  |  Suc: 0.00  |  crash: 0.00  |  Max R: 69.0  |  var: 0.64  |  lr:1.00e-4
Ep: 799 --- Ave R: 25.5  |  Suc: 0.00  |  crash: 0.00  |  Max R: 58.0  |  var: 0.68  |  lr:1.00e-4


KeyboardInterrupt: 

## 